# LSTM Baseline Model Testing

## 0 Imports & Constants

In [1]:
import sys
import os

# Füge das übergeordnete Verzeichnis zu sys.path hinzu'
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as dc
from sklearn.preprocessing import MinMaxScaler
import itertools
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from utils.utils import load_time_series, add_lagged_data, scale_data, train_test_split_to_tensor, inverse_scale_data

from baseline_model.TimeSeriesDataset import TimeSeriesDataset
from baseline_model.LSTM import LSTM, train_model

In [3]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.3.1+cu121
12.1
True
NVIDIA TITAN RTX


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
DATA_FOLDER = Path("../data")
MULTIVARIATE_DATA_FOLDER = DATA_FOLDER / "multivariate"
UNIVARIATE_DATA_FOLDER = DATA_FOLDER / "univariate"
BENCHMARK = False

## 1 Data

### Data Loading

In [6]:
# Load data from csv
# -> convert Date column to datetime
data = load_time_series(f'{UNIVARIATE_DATA_FOLDER}/NVDA_open_high_low_close_adjClose_volume_99_24.csv')

## 2 Benchmark Loop

In [7]:
hyperparameters = {
    'lag': [7, 14, 21],
    'lr': [0.002, 0.001],
    'hidden_size': [2, 4, 6, 8, 12],
    'num_layers': [1, 2],
    'batch_size': [64, 8, 16, 32],
}

In [8]:
# get all combinations of hyperparameters
keys, values = zip(*hyperparameters.items())
possible_hyperparameters = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [9]:
possible_features = [
    ['Close', 'Volume'],
    ['Close', 'Open', 'Volume'],
    ['Close', 'Open', 'High', 'Low', 'Volume']
]

In [10]:
RESULTS = {}
for features in possible_features:

    for hyperparameters in tqdm(possible_hyperparameters):
        
        print('*'*50)
        print('*'*50)
        print('STARTING NEW BENCHMARK RUN:')
        print(f"Features: {features}")
        print(f"Hyperparameters: {hyperparameters}")
        print('*'*50)
        print('*'*50)

        ### Select features
        features_incl_date = features+['Date']
        data_only_features = data[features_incl_date]
        
        ### Data Preprocessing
        data_lagged = add_lagged_data(data_only_features, hyperparameters['lag'], features)
        data_lagged_scaled, scaler_close = scale_data(data_lagged)
        X_train, y_train, X_test, y_test = train_test_split_to_tensor(data_lagged_scaled)

        ### Create datasets and DataLoaders
        train_dataset = TimeSeriesDataset(X_train, y_train)
        test_dataset = TimeSeriesDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, pin_memory=True, num_workers=8)
        test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, pin_memory=True, num_workers=8)

        ### Train model
        validation_losses = [] # reset validation losses
        for i in range(2): # train 2 times because sometimes the model converges to a local minimum
            ### Instantiate model
            model = LSTM(
                device=device,
                input_size=len(features),
                hidden_size=hyperparameters['hidden_size'],
                num_stacked_layers=hyperparameters['num_layers']
            ).to(device)

            ### Optimizer, Criterion
            optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters['lr'])
            criterion = nn.MSELoss()

            validation_loss, model = train_model(
                model=model,
                train_loader=train_loader,
                test_loader=test_loader,
                optimizer=optimizer,
                criterion=criterion,
                device=device)
            
            # save loss for each run
            validation_losses.append(validation_loss)
        
        ### Save results
        feature_acronym = ''.join([feature[0] for feature in features])
        RESULTS[f'{feature_acronym}_lag{hyperparameters["lag"]}_lr{hyperparameters["lr"]}_bs{hyperparameters["batch_size"]}_hs{hyperparameters["hidden_size"]}_nl{hyperparameters["num_layers"]}'] = min(validation_losses)

print('#'*50)
print('#'*50)
print('#'*50)
print('TRAINING FINISHED')
print('#'*50)
print('#'*50)
print('#'*50)

  0%|          | 0/240 [00:00<?, ?it/s]

**************************************************
**************************************************
STARTING NEW BENCHMARK RUN:
Features: ['Close', 'Volume']
Hyperparameters: {'lag': 7, 'lr': 0.002, 'hidden_size': 2, 'num_layers': 1, 'batch_size': 64}
**************************************************
**************************************************
Adding lagged data for columns: ['Close', 'Volume']
Shape of the numpy array wit lagged data: (6384, 8, 2)
Shape of X_train: torch.Size([6064, 7, 2]) 
 Shape of y_train: torch.Size([6064, 1]) 
 Shape of X_test: torch.Size([320, 7, 2]) 
 Shape of y_test: torch.Size([320, 1])
Epoch: 1
Validation Loss: 0.08576316554099321
**************************************************
Epoch: 2
Validation Loss: 0.05997884273529053
**************************************************
Epoch: 3
Validation Loss: 0.048535099253058434
**************************************************
Epoch: 4
Validation Loss: 0.04020353402011097
*******************************

  0%|          | 0/240 [00:22<?, ?it/s]


KeyboardInterrupt: 

## 3 Evaluate Results

In [ ]:
RESULTS

{'CV_lag7_lr0.002_bs4_hs2_nl1': 0.003400171762939408,
 'CV_lag7_lr0.002_bs8_hs2_nl1': 0.010209126127210765,
 'CV_lag7_lr0.002_bs16_hs2_nl1': 0.005297926136427123,
 'CV_lag7_lr0.002_bs32_hs2_nl1': 0.0056508751447836405,
 'CV_lag7_lr0.002_bs4_hs2_nl2': 0.014161826070790084,
 'CV_lag7_lr0.002_bs8_hs2_nl2': 0.012155452139359113,
 'CV_lag7_lr0.002_bs16_hs2_nl2': 0.017698677682892593}

In [ ]:
best_params = min(RESULTS, key=RESULTS.get)
best_params

'CV_lag7_lr0.002_bs4_hs2_nl1'

In [ ]:
RESULTS[best_params]

0.003400171762939408